In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [8]:
df= pd.read_csv(r"C:\Users\barta\Documents\cours\iaprojets\projet_aa\repo\Projet-AARES-20260202\part_1\data\raw\recommendation_spotify.csv")

In [9]:
df.head()

,acousticness,artists,danceability,duration_ms,energy,explicit,id,instrumentalness,key,liveness,loudness,mode,name,popularity,release_date,speechiness,tempo,valence,year
0,0.991000,['Mamie Smith'],0.598,168333,0.224,0,0cS0A1fUEUd1EW3FcF8AEI,0.000522,5,0.3790,-12.628,0,Keep A Song In Your Soul,12,1920,0.0936,149.976,0.6340,1920
1,0.643000,"[""Screamin' Jay Hawkins""]",0.852,150200,0.517,0,0hbkKFIJm7Z05H8Zl9w30f,0.026400,5,0.0809,-7.261,0,I Put A Spell On You,7,1920-01-05,0.0534,86.889,0.9500,1920
2,0.993000,['Mamie Smith'],0.647,163827,0.186,0,11m7laMUgmOKqI3oYzuhne,0.000018,0,0.5190,-12.098,1,Golfing Papa,4,1920,0.1740,97.600,0.6890,1920
3,0.000173,['Oscar Velazquez'],0.730,422087,0.798,0,19Lc5SfJJ5O1oaxY0fpwfh,0.801000,2,0.1280,-7.311,1,True House Music - Xavier Santos & Carlos Gomi...,17,1920-01-01,0.0425,127.997,0.0422,1920
4,0.295000,['Mixe'],0.704,165224,0.707,1,2hJjbsLCytGsnAHfdsLejp,0.000246,10,0.4020,-6.036,0,Xuniverxe,2,1920-10-01,0.0768,122.076,0.2990,1920


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174389 entries, 0 to 174388
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   acousticness      174389 non-null  float64
 1   artists           174389 non-null  object 
 2   danceability      174389 non-null  float64
 3   duration_ms       174389 non-null  int64  
 4   energy            174389 non-null  float64
 5   explicit          174389 non-null  int64  
 6   id                174389 non-null  object 
 7   instrumentalness  174389 non-null  float64
 8   key               174389 non-null  int64  
 9   liveness          174389 non-null  float64
 10  loudness          174389 non-null  float64
 11  mode              174389 non-null  int64  
 12  name              174389 non-null  object 
 13  popularity        174389 non-null  int64  
 14  release_date      174389 non-null  object 
 15  speechiness       174389 non-null  float64
 16  tempo             17

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
import ast


class MusicFeatureEngineering():
    def __init__(self):
        self.min_date_ = None

    def fit(self, X, y=None):
        self.min_date_ =  pd.Timestamp('1900-01-01')
        return self

    def transform(self, X):
        X_copy = X.copy()
        
        X_copy['release_date'] = pd.to_datetime(X_copy['release_date'], format='mixed')
        X_copy['days_since_1st_date'] = (X_copy['release_date'] - self.min_date_).dt.days

        cols_to_drop = ['release_date','id','name','year']
        X_copy = X_copy.drop(columns=[c for c in cols_to_drop if c in X_copy.columns])
        numeric_features = X_copy.select_dtypes(include=[np.number]).columns.tolist()
        print (numeric_features)
        categorical_features = X_copy.select_dtypes(include=['object', 'category','bool']).columns.tolist()
        # Encoder chaque feature catégorique
        label_encoders = {}
        for col in categorical_features:
            le = LabelEncoder()
            X_copy[col] = le.fit_transform(X_copy[col].astype(str))
            label_encoders[col] = le
        return X_copy

X = df

pipeline_X = Pipeline([
    ('feature_eng', MusicFeatureEngineering())])

X_processed = pipeline_X.fit_transform(X)
X_processed.head()

['acousticness', 'danceability', 'duration_ms', 'energy', 'explicit', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'popularity', 'speechiness', 'tempo', 'valence', 'days_since_1st_date']


,acousticness,artists,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,loudness,mode,popularity,speechiness,tempo,valence,days_since_1st_date
0,0.991000,21071,0.598,168333,0.224,0,0.000522,5,0.3790,-12.628,0,12,0.0936,149.976,0.6340,7304
1,0.643000,303,0.852,150200,0.517,0,0.026400,5,0.0809,-7.261,0,7,0.0534,86.889,0.9500,7308
2,0.993000,21071,0.647,163827,0.186,0,0.000018,0,0.5190,-12.098,1,4,0.1740,97.600,0.6890,7304
3,0.000173,24517,0.730,422087,0.798,0,0.801000,2,0.1280,-7.311,1,17,0.0425,127.997,0.0422,7304
4,0.295000,22782,0.704,165224,0.707,1,0.000246,10,0.4020,-6.036,0,2,0.0768,122.076,0.2990,7578


In [ ]:
X_processed.shape # parfait, j'ai retiré 'id','name','year'

(174389, 16)

In [22]:
print(X_processed['popularity'].value_counts(),
40905/174389)

popularity
0      40905
1       3846
33      2819
34      2790
36      2784
       ...  
94         5
96         2
93         2
95         2
100        1
Name: count, Length: 98, dtype: int64 0.23456181295838613


### 1. Propose a method that returns a playlist (approx. 10 songs) given a song (or a list of songs).

In [ ]:
# Im going to implement a knn algo
from sklearn.cluster import KMeans
def playlist(songs):
    """enter list of songs described by its name or id 
    returns a list of songs of lenght 10
     """

    n_clusters = 10
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    clusters = kmeans.fit_predict(X_processed[songs])
